# Argos Query Suggestions

In [1]:
import requests
import json
from fake_useragent import UserAgent

In [2]:
ua = UserAgent()
#headers = {"user-agent": ua.chrome}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

In [3]:
import pandas as pd
import numpy as np

### Loading list of toys

In [4]:
with open('all_items.txt') as f:
    contents = f.read().splitlines()
contents

['vehicle toys',
 'sport',
 'military toys',
 'race cars',
 'outer space toys',
 'depots',
 'machines',
 'doll-humanoid',
 'action figures',
 'gi joe action figure',
 'spiderman',
 'guitar',
 'castle tent',
 'microscope',
 'monster trucks',
 'bug collection set',
 'barn',
 'grill',
 'dinosaur toy',
 'volcano creator',
 'pokemon cards',
 'space station toys',
 'basketball hoop',
 'gears',
 'train set',
 'lincoln logs',
 'police station toy',
 'airport toy',
 'police officer gear',
 'dragonballz',
 'superhero costumes',
 'helicopter',
 'racetrack',
 'remote control car',
 'football',
 'hockey stick',
 'sword toy',
 'miniature guns',
 'truck set',
 'remote control helicopter',
 'sully costume',
 'woody valentine',
 'spiderman costume',
 'batman costume',
 'ninja costume',
 'wwe action figure',
 'action figure',
 'toy gun',
 'construction games',
 'war toys',
 'toy vehicles',
 'educational material',
 'nerf guns',
 'construction toys',
 'car toys',
 'video games',
 'building blocks',
 'din

## Loading pre-documented gender stereotyped toys

In [5]:
stereo_toys = pd.read_csv('./predoc_stereotyped_items.csv', delimiter =',')
stereo_toys

,BOY,GIRL,NEUTRAL
0,vehicle toys,doll,toy animals
1,sport,domestic toys,books
2,military toys,educational art,educational teaching
3,race cars,clothes,musical games
4,outer space toys,dollhouses,games
...,...,...,...
67,toy rocket,barbie furniture set,NaN
68,soccer ball,NaN,NaN
69,blue ipad,pink ipad,NaN
70,toy robots,NaN,NaN


### Some stat on the dataset

In [6]:
boy_toys = stereo_toys['BOY'].unique().tolist()
len(boy_toys)

71

In [7]:
girl_toys = stereo_toys['GIRL'].unique().tolist()
len(girl_toys)

64

In [8]:
neutral_toys = stereo_toys['NEUTRAL'].unique().tolist()
len(neutral_toys)

33

In [9]:
all_items = boy_toys + girl_toys + neutral_toys
len(all_items)

168

In [10]:
all_items = [x for x in all_items if isinstance(x, str)]

## Preparing the data for query

### Adding "for"

In [11]:
new_content = [x.strip() for x in all_items]
new_content = [x.replace(' for','') for x in new_content]
new_content = [x.lower() for x in new_content]
new_content

['vehicle toys',
 'sport',
 'military toys',
 'race cars',
 'outer space toys',
 'depots',
 'machines',
 'doll-humanoid',
 'action figures',
 'gi joe action figure',
 'spiderman',
 'guitar',
 'castle tent',
 'microscope',
 'monster trucks',
 'bug collection set',
 'barn',
 'grill',
 'dinosaur toy',
 'volcano creator',
 'pokemon cards',
 'space station toys',
 'basketball hoop',
 'gears',
 'train set',
 'lincoln logs',
 'police station toy',
 'airport toy',
 'police officer gear',
 'dragonballz',
 'superhero costumes',
 'helicopter',
 'racetrack',
 'remote control car',
 'football',
 'hockey stick',
 'sword toy',
 'miniature guns',
 'truck set',
 'remote control helicopter',
 'sully costume',
 'woody valentine',
 'spiderman costume',
 'batman costume',
 'ninja costume',
 'wwe action figure',
 'action figure',
 'toy gun',
 'construction games',
 'war toys',
 'toy vehicles',
 'educational material',
 'nerf guns',
 'construction toys',
 'car toys',
 'video games',
 'building blocks',
 'din

In [12]:
search_terms = [x+' for' for x in new_content]
search_terms

['vehicle toys for',
 'sport for',
 'military toys for',
 'race cars for',
 'outer space toys for',
 'depots for',
 'machines for',
 'doll-humanoid for',
 'action figures for',
 'gi joe action figure for',
 'spiderman for',
 'guitar for',
 'castle tent for',
 'microscope for',
 'monster trucks for',
 'bug collection set for',
 'barn for',
 'grill for',
 'dinosaur toy for',
 'volcano creator for',
 'pokemon cards for',
 'space station toys for',
 'basketball hoop for',
 'gears for',
 'train set for',
 'lincoln logs for',
 'police station toy for',
 'airport toy for',
 'police officer gear for',
 'dragonballz for',
 'superhero costumes for',
 'helicopter for',
 'racetrack for',
 'remote control car for',
 'football for',
 'hockey stick for',
 'sword toy for',
 'miniature guns for',
 'truck set for',
 'remote control helicopter for',
 'sully costume for',
 'woody valentine for',
 'spiderman costume for',
 'batman costume for',
 'ninja costume for',
 'wwe action figure for',
 'action figur

## Search Engines and E-Com Sites

### Argos

In [65]:
from urllib.request import urlopen, Request
import urllib.parse as urlparse
from bs4 import BeautifulSoup
#custom_user_agent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
#headers={"User-Agent": custom_user_agent}

def argos_scrape(query):
   # params = {"qry": query}
    url_parse = urlparse.urlparse('https://www.argos.co.uk/suggest?term={query}%20f&fuzziness=true&size=5&includeFaq=true')
    query = url_parse.query
    url_dict = dict(urlparse.parse_qsl(query))
    # url_dict.update(params)
    url_new_query = urlparse.urlencode(url_dict)
    url_parse = url_parse._replace(query=url_new_query)
    new_url = urlparse.urlunparse(url_parse)
    req = Request(new_url, headers=headers)
    page = urlopen(req)
    soup = BeautifulSoup(page.read())
    return soup

In [30]:
def argos_scrape(query):
    url = f'https://www.argos.co.uk/suggest?term={query}%20f&fuzziness=true&size=5&includeFaq=true'
    # with open(url) as j:
    #    response = json.loads(j.read())
    response = requests.get(url, headers=headers, verify=False).json()
    return response
#['autoSuggest']['keywords']

In [27]:
x = argos_scrape('plants')

In [28]:
for s in x:
    print(s['value'])

plant pot
planter
plant
plants
planters


### All together

In [17]:
def argos_auto(item):
    res = argos_scrape(item)
    suggestions = res['autoSuggest']['keywords']
    results = []
    for s in suggestions:
        results.append(s['value'])
    return results

In [32]:
argos_auto('plants')

['plant pot', 'planter', 'plant', 'plants', 'planters']

## Run queries

In [31]:
import pandas as pd
columns = ['platform', 'query', 'suggestions']
df = pd.DataFrame(columns = columns)
df

,platform,query,suggestions


In [19]:
def search(platform):
    if platform == 'Argos':
        return argos_auto(item)

In [33]:
trial = ['plants','toys','doll']

In [34]:
columns = ['query', 'suggestions']

In [36]:
# our version 
import time
import warnings
warnings.filterwarnings('ignore')
data = []
for item in contents:
    result = argos_auto(item)
    values = [item, result]
    zipped = zip(columns, values)
    a_dictionary = dict(zipped)
    print(a_dictionary)
    time.sleep(1.5)
    data.append(a_dictionary)

{'query': 'vehicle toys', 'suggestions': ['vehicle toy', 'building blocks vehicle toy', 'construction digger excavator vehicle toy', 'friends treeplanting vehicle toy', 'esports vehicle toy']}
{'query': 'sport', 'suggestions': ['nintendo switch sports', 'switch sports', 'sports', 'sports watch', 'sport']}
{'query': 'military toys', 'suggestions': []}
{'query': 'race cars', 'suggestions': ['race car', 'race car set', 'lego ninjago lloyds race car', 'race car model set 42141', 'bluey hopscotch race card game']}
{'query': 'outer space toys', 'suggestions': []}
{'query': 'depots', 'suggestions': []}
{'query': 'machines', 'suggestions': ['washing machine', 'coffee machine', 'sewing machine', 'karaoke machine', 'rowing machine']}
{'query': 'doll-humanoid', 'suggestions': []}
{'query': 'action figures', 'suggestions': ['action figures', 'action figure', 'pop action figure', 'batman action figure', 'jackobonnie fnaf dreadbear action figure']}
{'query': 'gi joe action figure', 'suggestions': []

{'query': 'tea set', 'suggestions': ['tea set', 'afternoon tea set', 'wooden tea set', 'chad valley wooden tea set', 'frozen tea set']}
{'query': 'vanity set', 'suggestions': ['vanity set', 'chad valley wooden vanity set', 'wooden vanity set', 'valley wooden vanity set']}
{'query': 'princess costume', 'suggestions': []}
{'query': 'baby dolls', 'suggestions': ['baby doll', 'baby dolls', 'baby dolls carrier', 'crawling baby doll', 'boy baby doll']}
{'query': 'beads', 'suggestions': ['double bed frame', 'single bed frame', 'kingsize bed frame', 'small double bed frame', 'bed frame']}
{'query': 'dollhouse', 'suggestions': ['gabbys dollhouse figure', 'gabbys dollhouse figure gift pack', 'dollhouse figure', 'dolls house', 'doll house']}
{'query': 'easy bake oven', 'suggestions': []}
{'query': 'baby doll stroller', 'suggestions': []}
{'query': 'sewing machine', 'suggestions': ['sewing machine', 'sewing machine carry bag', 'singer 1306 sewing machine', 'computerised sewing machine', 'stitch se

In [20]:
import time
import warnings
warnings.filterwarnings('ignore')
platforms = ['Argos']
data = []
for item in contents:
    for platform in platforms:
        result = search(platform) 
        values = [platform, item, result]
        zipped = zip(columns, values)
        a_dictionary = dict(zipped)
        #print(a_dictionary)
        time.sleep(1.5)
        data.append(a_dictionary)

KeyboardInterrupt: 

In [37]:
data

[{'query': 'vehicle toys',
  'suggestions': ['vehicle toy',
   'building blocks vehicle toy',
   'construction digger excavator vehicle toy',
   'friends treeplanting vehicle toy',
   'esports vehicle toy']},
 {'query': 'sport',
  'suggestions': ['nintendo switch sports',
   'switch sports',
   'sports',
   'sports watch',
   'sport']},
 {'query': 'military toys', 'suggestions': []},
 {'query': 'race cars',
  'suggestions': ['race car',
   'race car set',
   'lego ninjago lloyds race car',
   'race car model set 42141',
   'bluey hopscotch race card game']},
 {'query': 'outer space toys', 'suggestions': []},
 {'query': 'depots', 'suggestions': []},
 {'query': 'machines',
  'suggestions': ['washing machine',
   'coffee machine',
   'sewing machine',
   'karaoke machine',
   'rowing machine']},
 {'query': 'doll-humanoid', 'suggestions': []},
 {'query': 'action figures',
  'suggestions': ['action figures',
   'action figure',
   'pop action figure',
   'batman action figure',
   'jackobon

In [40]:
df = df.append(data, True)
df['platform'] = 'Argos'
df

,platform,query,suggestions
0,Argos,vehicle toys,"[vehicle toy, building blocks vehicle toy, con..."
1,Argos,sport,"[nintendo switch sports, switch sports, sports..."
2,Argos,military toys,[]
3,Argos,race cars,"[race car, race car set, lego ninjago lloyds r..."
4,Argos,outer space toys,[]
...,...,...,...
493,Argos,scooter,"[scooter, electric scooter, stunt scooter, mob..."
494,Argos,drum set,"[drum set, electronic drum set, chad valley el..."
495,Argos,puzzles,"[jigsaw puzzle, puzzle, puzzles, jigsaw puzzle..."
496,Argos,board games,"[board games, board game, kids board game, fam..."


In [41]:
df.to_csv('argos_query_suggestions.csv', index = False)